In [1]:
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data
import csv
import numpy as np

DATA_DIR = 'MNIST'
sentiment_data = 'sentiment-data'


def getWordVectorDict():
    reader = csv.reader(open(sentiment_data +'/word-vectors-refine.txt'))

    word_vector_dict = {}
    for row in reader:
        key = row[0]
        if key in word_vector_dict:
            # implement your duplicate row handling here
            pass
        word_vector_dict[key] = np.array(row[1:])
    return word_vector_dict

def getPaddedSentenceMatrix(sentenceMatrix):
    wordCount = 100
    return np.vstack((sentenceMatrix, np.zeros((wordCount - np.shape(sentenceMatrix)[0],np.shape(sentenceMatrix)[1]), dtype=np.float32)))

def getVectorForSentence(sentence, word_vec_dict):
    sentence_matrix = []
    for word in sentence.split(' '):
        word_vec = word_vec_dict[word]
        if(len(sentence_matrix) == 0):
            sentence_matrix = word_vec
        else:
            sentence_matrix = np.vstack((sentence_matrix,word_vec))
    return getPaddedSentenceMatrix(sentence_matrix)

def getData(fileName):
    reader = csv.reader(open(sentiment_data +'/' + fileName))
    trainingData = []
    for row in reader:
        data = {}
        data['label'] =  1 if row[0] == 'postive' else 0
        data['sentence'] = row[1:]
        trainingData.append(data)
    return trainingData

word_vec_dict = getWordVectorDict()


def transform(row):
    return row['label'], getVectorForSentence(row['sentence'][0], word_vec_dict)


word_vector_size = 50;
time_steps = 100;
num_classes = 2
batch_size = 1000;
n_iterations = 10;
hidden_layer_size = 50

training_data = getData('train.csv')
training_rows  = map(lambda row: transform(row), training_data)
training_data = map(lambda row: row[1], training_rows)
training_labels = map(lambda row: row[0], training_rows)
#test_data = map(lambda row: transform(row), getData('test.csv'))
test_data = getData('test.csv')
test_rows  = map(lambda row: transform(row), test_data)
test_data = map(lambda row: row[1], test_rows)
test_labels = map(lambda row: row[0], test_rows)
print training_data[0]

[['0.68491' '0.32385' '-0.11592' ..., '0.17874' '-0.1693' '0.062375']
 ['0.96193' '0.012516' '0.21733' ..., '0.14032' '-0.38468' '-0.38712']
 ['0.6008' '0.18044' '0.078339' ..., '-0.016404' '-0.65372' '-0.38255']
 ..., 
 ['0.88387' '-0.14199' '0.13566' ..., '0.52711' '-0.20148' '0.0095952']
 ['-0.0010919' '0.33324' '0.35743' ..., '-0.45697' '-0.048969' '1.1316']
 ['-0.55114' '-0.16296' '-0.95494' ..., '-1.0346' '-0.25143' '1.4836']]


In [2]:

#Setting up the input and labels placeholders
_inputs = tf.placeholder(tf.float32, shape=[None, time_steps,
                                            word_vector_size])
y = tf.placeholder(tf.int32, shape=[None, num_classes])
y_one_hot = tf.one_hot( y , num_classes )

# TensorFlow built-in functions
# Creating the RNN cell and creating the outputs
with tf.variable_scope("lstm"):
    lstm_cell = tf.contrib.rnn.BasicLSTMCell(hidden_layer_size,
                                             forget_bias=1.0)
    outputs, states = tf.nn.dynamic_rnn(lstm_cell,_inputs, dtype=tf.float32)

weights = {
        'linear_layer': tf.Variable(tf.truncated_normal([hidden_layer_size,
                                                         num_classes],
                                                         mean=0,stddev=.01))
}


# Extract the last relevant output and use in a linear layer
final_output = tf.matmul(states[1],
                         weights["linear_layer"])
softmax = tf.nn.softmax_cross_entropy_with_logits(logits = final_output,
                                                  labels = y)                         
cross_entropy = tf.reduce_mean(softmax)

train_step = tf.train.RMSPropOptimizer(0.005, 0.9,0.01).minimize(cross_entropy)
correct_prediction = tf.equal(tf.argmax(y,1),
                              tf.argmax(final_output,1))
accuracy = (tf.reduce_mean(tf.cast(correct_prediction,
                                   tf.float32)))*100






In [3]:
def dense_to_one_hot(labels_dense, num_classes=10):
  """Convert class labels from scalars to one-hot vectors."""
  num_labels = labels_dense.shape[0]
  index_offset = np.arange(num_labels) * num_classes
  labels_one_hot = np.zeros((num_labels, num_classes))
  labels_one_hot.flat[index_offset + labels_dense.ravel()] = 1
  return labels_one_hot

label_batch = np.array(training_labels[100 : 200])
l = np.array(dense_to_one_hot(label_batch,2)).reshape(len(label_batch), num_classes)

print l.shape

(100, 2)


In [4]:
batch_size = 100;
n_iterations = 20;

#Initialize session
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())

    #Initialize test data
    #test_data = mnist.test.images[:batch_size].reshape((-1,time_steps, word_vector_size))
    #test_label = mnist.test.labels[:batch_size]

    for epoch in range(n_iterations):
        print "Epoch:", epoch
        for j in range(len(training_data)/batch_size):
            #print "j:", j
            if(j == 100):
                break
            startIndex = j*batch_size
            endIndex = startIndex + batch_size
            batch_x = np.array(training_data[startIndex : endIndex]).reshape((-1,time_steps, word_vector_size))
            #print len(training_labels[startIndex : endIndex])
            batch_y = dense_to_one_hot(np.array(training_labels[startIndex : endIndex]),num_classes)
            #print "X:", len(batch_x)
            #print "Y:", len(batch_y)
            #batch_x, batch_y = mnist.train.next_batch(batch_size)
            #batch_x = batch_x.reshape((batch_size, time_steps, word_vector_size))
            sess.run(train_step, feed_dict={_inputs: batch_x,
                                            y: batch_y})
            if epoch % 1 == 0:
                acc = sess.run(accuracy, feed_dict={_inputs: batch_x, y: batch_y})
                loss = sess.run(cross_entropy, feed_dict={_inputs: batch_x, y: batch_y})
                print ("Iter " + str(epoch) + ", Minibatch Loss= " + \
                       "{:.6f}".format(loss) + ", Training Accuracy= " + \
                       "{:.5f}".format(acc))


    print ("Testing Accuracy:",
        sess.run(accuracy, feed_dict={_inputs: test_data, y: dense_to_one_hot(np.array(test_labels), num_classes)}))

Epoch: 0
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693461, Training Accuracy= 48.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692917, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692556, Training Accuracy= 55.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692934, Training Accuracy= 53.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692942, Training Accuracy= 52.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693424, Training Accuracy= 45.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693386, Training Accuracy= 49.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692397, Training Accuracy= 55.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693163, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692959, Training Accuracy= 56.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.693013, Training Accuracy= 48.00000
100
X: 100
Y: 100
Iter 0, Minibatch Loss= 0.692739, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 0, Minib

Iter 1, Minibatch Loss= 0.691238, Training Accuracy= 53.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.691028, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.692887, Training Accuracy= 44.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.691343, Training Accuracy= 52.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.693423, Training Accuracy= 51.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.689382, Training Accuracy= 66.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.692336, Training Accuracy= 50.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.690685, Training Accuracy= 62.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.690483, Training Accuracy= 58.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.690878, Training Accuracy= 58.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.690982, Training Accuracy= 53.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.689376, Training Accuracy= 54.00000
100
X: 100
Y: 100
Iter 1, Minibatch Loss= 0.689219, Traini

Iter 2, Minibatch Loss= 0.665891, Training Accuracy= 56.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.694913, Training Accuracy= 54.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.612035, Training Accuracy= 66.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.661540, Training Accuracy= 61.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.613033, Training Accuracy= 69.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.571754, Training Accuracy= 78.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.630923, Training Accuracy= 66.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.621718, Training Accuracy= 71.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.603904, Training Accuracy= 69.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.611084, Training Accuracy= 67.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.626574, Training Accuracy= 67.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.622319, Training Accuracy= 61.00000
100
X: 100
Y: 100
Iter 2, Minibatch Loss= 0.670457, Traini

Iter 3, Minibatch Loss= 0.617629, Training Accuracy= 68.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.560818, Training Accuracy= 73.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.533543, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.609249, Training Accuracy= 68.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.583608, Training Accuracy= 70.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.555367, Training Accuracy= 72.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.515433, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.550708, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.568534, Training Accuracy= 72.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.602704, Training Accuracy= 66.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.604237, Training Accuracy= 70.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.625899, Training Accuracy= 65.00000
100
X: 100
Y: 100
Iter 3, Minibatch Loss= 0.578739, Traini

Iter 4, Minibatch Loss= 0.579937, Training Accuracy= 70.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.550144, Training Accuracy= 73.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.634687, Training Accuracy= 68.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.514708, Training Accuracy= 75.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.528437, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.602011, Training Accuracy= 61.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.590567, Training Accuracy= 70.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.575224, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.654321, Training Accuracy= 64.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.613482, Training Accuracy= 64.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.533724, Training Accuracy= 74.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.523096, Training Accuracy= 76.00000
100
X: 100
Y: 100
Iter 4, Minibatch Loss= 0.635006, Traini

...

Iter 15, Minibatch Loss= 0.398971, Training Accuracy= 82.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.395932, Training Accuracy= 81.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.439543, Training Accuracy= 82.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.407159, Training Accuracy= 81.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.487108, Training Accuracy= 81.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.388624, Training Accuracy= 80.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.441070, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.445113, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.342183, Training Accuracy= 89.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.441652, Training Accuracy= 78.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.352196, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.418134, Training Accuracy= 82.00000
100
X: 100
Y: 100
Iter 15, Minibatch Loss= 0.4

Iter 16, Minibatch Loss= 0.433023, Training Accuracy= 80.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.383999, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.457600, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.368386, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.472666, Training Accuracy= 76.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.452919, Training Accuracy= 81.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.350339, Training Accuracy= 88.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.457591, Training Accuracy= 76.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.385599, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.362955, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.427113, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.432702, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 16, Minibatch Loss= 0.3

Iter 17, Minibatch Loss= 0.493805, Training Accuracy= 80.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.402687, Training Accuracy= 81.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.440828, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.445901, Training Accuracy= 81.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.356005, Training Accuracy= 88.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.429054, Training Accuracy= 79.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.335347, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.371107, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.435474, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.419459, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.357694, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.511709, Training Accuracy= 71.00000
100
X: 100
Y: 100
Iter 17, Minibatch Loss= 0.4

X: 100
Y: 100
Iter 18, Minibatch Loss= 0.418886, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.428267, Training Accuracy= 82.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.354427, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.465212, Training Accuracy= 77.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.369221, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.359925, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.425175, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.394961, Training Accuracy= 82.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.346124, Training Accuracy= 86.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.492901, Training Accuracy= 71.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.456638, Training Accuracy= 81.00000
100
X: 100
Y: 100
Iter 18, Minibatch Loss= 0.420561, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 18, Minib

Iter 19, Minibatch Loss= 0.414197, Training Accuracy= 82.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.361314, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.433457, Training Accuracy= 78.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.310076, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.340049, Training Accuracy= 84.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.427490, Training Accuracy= 87.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.399782, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.363996, Training Accuracy= 85.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.458809, Training Accuracy= 76.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.429374, Training Accuracy= 81.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.396371, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.408708, Training Accuracy= 83.00000
100
X: 100
Y: 100
Iter 19, Minibatch Loss= 0.3